In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `Mehrab_1` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/h

In [ ]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 39.7 MB/s eta 0:00:00


In [ ]:
import os, gc, cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers, callbacks
from datasets import load_dataset
from decord import VideoReader, cpu

#Load_data

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nexar-ai/nexar_collision_prediction")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/1502 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1348 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['video', 'time_of_event', 'time_of_alert', 'light_conditions', 'weather', 'scene', 'time_to_accident'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['video', 'time_of_event', 'time_of_alert', 'light_conditions', 'weather', 'scene', 'time_to_accident'],
        num_rows: 1344
    })
})

#Preprocess

In [ ]:
# مسیر cache دیتاست
base_path = "/root/.cache/huggingface/hub/datasets--nexar-ai--nexar_collision_prediction/snapshots"
snapshot_dir = [os.path.join(base_path, d) for d in os.listdir(base_path)][0]
print(" Dataset directory:", snapshot_dir)

 Dataset directory: /root/.cache/huggingface/hub/datasets--nexar-ai--nexar_collision_prediction/snapshots/aa97deda5a59f00bb7187739053b7c72e14374df


In [ ]:
def build_dataframe(base_dir):
    positive_videos = glob.glob(f"{base_dir}/train/positive/*.mp4")
    negative_videos = glob.glob(f"{base_dir}/train/negative/*.mp4")
    data = [{"video_path": p, "label": 1} for p in positive_videos] + \
           [{"video_path": p, "label": 0} for p in negative_videos]
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} videos (Pos:{len(positive_videos)}, Neg:{len(negative_videos)})")
    return df

In [ ]:
df = build_dataframe(snapshot_dir)

Loaded 1500 videos (Pos:750, Neg:750)


In [ ]:
train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=42
    )
print("Train:", len(train_df), "Val:", len(val_df))

Train: 1200 Val: 300


#Frame

In [ ]:
def load_video_frames(video_path, num_frames=32, size=(112,112), time_of_event=None):

    try:
        vr = VideoReader(video_path, ctx=cpu(0))
        total_frames = len(vr)
        fps = vr.get_avg_fps() if hasattr(vr, "get_avg_fps") else 30

        if time_of_event is not None and time_of_event > 0:
            event_frame = int(time_of_event * fps)
            start = max(event_frame - num_frames // 2, 0)
            end = min(event_frame + num_frames // 2, total_frames)
            indices = np.linspace(start, end - 1, num_frames, dtype=int)
        else:
            indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

        frames = vr.get_batch(indices).asnumpy()
        frames = np.array([cv2.resize(f, size) for f in frames])

        frames = frames.astype("float32") / 255.0
        return frames

    except Exception as e:
        print(f"Erorr {video_path}: {e}")
        return None

In [ ]:
def preprocess_split(df, num_videos=100, num_frames=32):

    X, y = [], []
    num_videos = min(num_videos, len(df))
    for i in tqdm(range(num_videos), desc="Extracting frames"):
        row = df.iloc[i]
        frames = load_video_frames(
            row["video_path"],
            num_frames=num_frames,
            time_of_event=row.get("time_of_event", None)
        )
        if frames is not None:
            X.append(frames)
            y.append(row["label"])
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = preprocess_split(train_df, num_videos=400)
X_val, y_val = preprocess_split(val_df, num_videos=100)

Extracting frames: 100%|██████████| 100/100 [07:12<00:00,  4.32s/it]


In [ ]:
print("Shapes:", X_train.shape, y_train.shape)
print("Shapes:", X_val.shape, y_val.shape)

Shapes: (400, 32, 112, 112, 3) (400,)
Shapes: (100, 32, 112, 112, 3) (100,)


In [ ]:
from collections import Counter
counter = Counter(y_train)
counter1 = Counter(y_val)
print(counter)
print(counter1)

Counter({np.int64(1): 201, np.int64(0): 199})
Counter({np.int64(1): 55, np.int64(0): 45})
